In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [4]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# Loading the Data

In [5]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [6]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [7]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_metr_la.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

In [8]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [9]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [10]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [11]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

In [ ]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

In [ ]:
i = 5
x_, y_ = x_test[i], y_test[i]

In [ ]:
from src.explanation.monte_carlo.explanation import get_instance_explanations


get_instance_explanations(
    x= x_,
    y = y_,
    distance_matrix = distance_matrix,
    spatial_temporal_gnn = spatial_temporal_gnn,
    scaler = scaler,
    n_rollouts= 50,
    explanation_size_factor = 2,
    cut_size_factor = 2,
    exploration_weight = 10,
    remove_value = 0.,
    verbose= True);

Execution 1/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 2/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 3/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 4/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 5/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 6/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 7/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 8/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 9/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 10/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 11/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 12/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 13/50
reward: 0.13933674139898833 , mae: 7.176857948303223
Execution 14/50
reward: 0.21191693215835242 , mae: 4.718830108642578
Execution 15/50
reward: 0.21191693215835242

In [ ]:
from src.explanation.monte_carlo.explanation import apply_grid_search

apply_grid_search(
    x_train[::10],
    y_train[::10],
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts_list=[30, 50],
    explanation_size_factor_list=[2, 3],
    cut_size_factor_list=[2],
    exploration_weight_list=[5, 10, 20],
    remove_value_list=[0.])

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: 0.0
[100/100] - 559s - MAE: { severe_congestion 3.01 -congestion 1.35 -free_flow 0.814 - total: 1.69 } - RMSE: { severe_congestion 3.62 -congestion 1.69 -free_flow 1.02 - total: 2.07 } - MAPE: { severe_congestion 13.4% -congestion 2.77% -free_flow 1.25% - total: 5.77% } - Average time: 5.59s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 50 remove_value: 0.0
[100/100] - 940s - MAE: { severe_congestion 3.12 -congestion 1.38 -free_flow 0.753 - total: 1.72 } - RMSE: { severe_congestion 3.78 -congestion 1.72 -free_flow 0.959 - total: 2.11 } - MAPE: { severe_congestion 14% -congestion 2.9% -free_flow 1.15% - total: 5.99% } - Average time: 9.4s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 10 n_rollouts: 30 remove_value: 0.0
[100/100] - 576s - MAE: { severe_congestion 3.49 -congestion 1.46 -free_flow 0.828 - total

In [ ]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 2
EXPLORATION_WEIGHT = 20
N_ROLLOUTS = 50

In [ ]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[999/999] - 10845s - MAE: { severe_congestion 3.32 -congestion 1.5 -free_flow 0.668 - total: 1.83 } - RMSE: { severe_congestion 4.07 -congestion 1.87 -free_flow 0.835 - total: 2.25 } - MAPE: { severe_congestion 14.9% -congestion 3.07% -free_flow 1.02% - total: 6.34% } - Average time: 10.9s 


In [30]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

Computing the explanations for the validation set...
[198/198] - 1871s - MAE: { severe_congestion 3.63 -congestion 1.39 -free_flow 0.668 - total: 1.86 } - RMSE: { severe_congestion 4.32 -congestion 1.73 -free_flow 0.848 - total: 2.26 } - MAPE: { severe_congestion 17.2% -congestion 2.75% -free_flow 1.02% - total: 6.88% } - Average time: 9.45s 


In [31]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

Computing the explanations for the test set...
[300/300] - 2663s - MAE: { severe_congestion 3.21 -congestion 1.65 -free_flow 0.713 - total: 1.84 } - RMSE: { severe_congestion 3.82 -congestion 2.06 -free_flow 0.881 - total: 2.24 } - MAPE: { severe_congestion 14.1% -congestion 3.45% -free_flow 1.09% - total: 6.14% } - Average time: 8.88s 


In [12]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train.npy'))

x_val_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val.npy'))

x_test_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))

In [22]:
from src.explanation.events import get_largest_event_set
from src.explanation.monte_carlo.explanation import evaluate
from src.spatial_temporal_gnn.metrics import MAE, RMSE, MAPE


def print_all_fidelity_plus(
    x,
    y,
    x_explained,
    spatial_temporal_gnn,
    remove_value=0.,
    divide_by_traffic_cluster_kind: bool = True):
    mae_criterion = MAE()
    rmse_criterion = RMSE()
    mape_criterion = MAPE()
    
    if divide_by_traffic_cluster_kind:
        # Get the severe congestion sparsity, the firs third of the data.
        severe_congestion_mae, severe_congestion_rmse, severe_congestion_mape = get_fidelity_plus(
            x[:len(x) // 3], 
            y[:len(x) // 3],
            x_explained[:len(x) // 3],  
            spatial_temporal_gnn, 
            mae_criterion,
            rmse_criterion,
            mape_criterion,
            remove_value)
        # Get the congestion sparsity, the second third of the data.
        congestion_mae, congestion_rmse, congestion_mape = get_fidelity_plus(
            x[len(x) // 3:2 * len(x) // 3],
            y[len(x) // 3:2 * len(x) // 3],
            x_explained[len(x) // 3:2 * len(x) // 3],
            spatial_temporal_gnn, 
            mae_criterion,
            rmse_criterion,
            mape_criterion,
            remove_value)
        # Get the free flow sparsity, the last third of the data.
        free_flow_mae, free_flow_rmse, free_flow_mape = get_fidelity_plus(
            x[2 * len(x) // 3:],
            y[2 * len(x) // 3:],
            x_explained[2 * len(x) // 3:],
            spatial_temporal_gnn, 
            mae_criterion,
            rmse_criterion,
            mape_criterion,
            remove_value)
        # Compute the average sparsity.
        mae = (severe_congestion_mae + congestion_mae +
                    free_flow_mae) / 3
        rmse = (severe_congestion_rmse + congestion_rmse +
                    free_flow_rmse) / 3
        mape = (severe_congestion_mape + congestion_mape +
                    free_flow_mape) / 3
        print(
            f'MAE+: {{ severe_congestion {severe_congestion_mae:.3g} -'
            f'congestion {congestion_mae:.3g} -'
            f'free_flow {free_flow_mae:.3g} -',
            f'total: {mae:.3g}% }} -'
            f'RMSE+: {{ severe_congestion {severe_congestion_rmse:.3g} -'
            f'congestion {congestion_rmse:.3g} -'
            f'free_flow {free_flow_rmse:.3g} -',
            f'total: {rmse:.3g}% }} -'
            f'MAPE+: {{ severe_congestion {severe_congestion_mape * 100:.3g}% -'
            f'congestion {congestion_mape * 100.:.3g}% -'
            f'free_flow {free_flow_mape * 100.:.3g}% -',
            f'total: {mape * 100.:.3g}% }}')
    else:
        print(
            f'MAE: {mae:.3g} -',
            f'RMSE: {rmse:.3g} -',
            f'MAPE: {mape * 100.:.3g}%')
        
def get_fidelity_plus(
    x, 
    y, 
    x_explained, 
    spatial_temporal_gnn, 
    mae_criterion,
    rmse_criterion,
    mape_criterion,
    remove_value):
    x_ = x.copy()
    # Get the events of the complement of x_explained.
    x_[x_explained != 0.] = 0.
    running_mae = 0.
    running_rmse = 0.
    running_mape = 0.
    for i in range(len(x_)):
        input_events = get_largest_event_set(x_[i])
    
        # Evaluate the results.
        mae, rmse, mape = evaluate(
            x_[i],
            y[i],
            input_events,
            spatial_temporal_gnn,
            scaler,
            mae_criterion,
            rmse_criterion,
            mape_criterion,
            remove_value)
        running_mae += mae
        running_rmse += rmse
        running_mape += mape

    return running_mae / len(x_), running_rmse / len(x_), running_mape / len(x_)

In [23]:
print('Computing the fidelity+ for the training set...')
print_all_fidelity_plus(x_train, y_train, x_train_explained, spatial_temporal_gnn, remove_value=0.)
print()

print('Computing the fidelity+ for the validation set...')
print_all_fidelity_plus(x_val, y_val, x_val_explained, spatial_temporal_gnn, remove_value=0.)
print()

print('Computing the fidelity+ for the test set...')
print_all_fidelity_plus(x_test, y_test, x_test_explained, spatial_temporal_gnn, remove_value=0.)
print()

Computing the fidelity+ for the training set...
MAE+: { severe_congestion 35.4 -congestion 7.96 -free_flow 2 - total: 15.1% } -RMSE+: { severe_congestion 36 -congestion 8.32 -free_flow 2.26 - total: 15.5% } -MAPE+: { severe_congestion 162% -congestion 16.6% -free_flow 3.06% - total: 60.4% }

Computing the fidelity+ for the validation set...
MAE+: { severe_congestion 37.5 -congestion 6.44 -free_flow 2.24 - total: 15.4% } -RMSE+: { severe_congestion 38 -congestion 6.77 -free_flow 2.49 - total: 15.8% } -MAPE+: { severe_congestion 194% -congestion 12.9% -free_flow 3.4% - total: 70.1% }

Computing the fidelity+ for the test set...
MAE+: { severe_congestion 35.3 -congestion 9.24 -free_flow 2.07 - total: 15.5% } -RMSE+: { severe_congestion 36 -congestion 9.69 -free_flow 2.31 - total: 16% } -MAPE+: { severe_congestion 165% -congestion 19.9% -free_flow 3.17% - total: 62.5% }



In [43]:
def print_all_sparsity(
    x,
    x_explained,
    divide_by_traffic_cluster_kind: bool = True):
    if divide_by_traffic_cluster_kind:
        # Get the severe congestion sparsity, the firs third of the data.
        severe_congestion_sparsity = get_sparsity(
            x[:len(x) // 3], x_explained[:len(x) // 3])
        # Get the congestion sparsity, the second third of the data.
        congestion_sparsity = get_sparsity(
            x[len(x) // 3:2 * len(x) // 3],
            x_explained[len(x) // 3:2 * len(x) // 3])
        # Get the free flow sparsity, the last third of the data.
        free_flow_sparsity = get_sparsity(
            x[2 * len(x) // 3:], x_explained[2 * len(x) // 3:])
        # Compute the average sparsity.
        sparsity = (severe_congestion_sparsity + congestion_sparsity +
                    free_flow_sparsity) / 3
        print(
            f'Sparsity: {{ severe_congestion {severe_congestion_sparsity:.3g} -'
            f'congestion {congestion_sparsity:.3g} -'
            f'free_flow {free_flow_sparsity:.3g} -',
            f'total: {sparsity:.3g} }}')
    else:
        print(f'Sparsity: {sparsity:.3g}')


def get_sparsity(x, x_explained):
    # Count the number of non-zero values in the original data, by time step in the last axis.
    x_non_zero = np.count_nonzero(x[..., 0], axis=0)
    # Count the number of non-zero values in the explained data, by time step.
    x_explained_non_zero = np.count_nonzero(x_explained[..., 0], axis=0)
    # Compute the sparsity, by time step.
    sparsity = 1 - x_explained_non_zero / x_non_zero
    # Compute the average sparsity.
    return np.mean(sparsity)

In [45]:
print('Computing the sparsity for the training set...')
print_all_sparsity(x_train, x_train_explained)
print()

print('Computing the sparsity for the validation set...')
print_all_sparsity(x_val, x_val_explained)
print()

print('Computing the sparsity for the test set...')
print_all_sparsity(x_test, x_test_explained)
print()

Computing the sparsity for the training set...
Sparsity: { severe_congestion 0.984 -congestion 0.988 -free_flow 0.985 - total: 0.986 }

Computing the sparsity for the validation set...
Sparsity: { severe_congestion 0.985 -congestion 0.988 -free_flow 0.985 - total: 0.986 }

Computing the sparsity for the test set...
Sparsity: { severe_congestion 0.983 -congestion 0.987 -free_flow 0.984 - total: 0.985 }

